# 22 - Inheritance

---

Inheritance allows you to create new classes based on existing once, just by indicating the difference. It is an extremely powerful concept that allows for the creation of highly flexible, easily maintainable programs.

---

## Class inheritance

In the previous chapters I gave the example of `Apple` and `Orange` both being subclasses of a class `Fruit`, and `Student` and `Teacher` both being subclasses of a class `Person`. You can implement such a hierarchy of classes and subclasses using "inheritance".

Basically, inheritance is really simple. When you define a new class, between parentheses you can specify another class. The new class inherits all the attributes and methods of the other class, i.e, they are automatically part of the new class.

In [ ]:
class Person:
    def __init__( self, firstname, lastname, age ):
        self.firstname = firstname
        self.lastname = lastname
        self.age = age
    def __repr__( self ):
        return "{} {}".format( self.firstname, self.lastname )
    def underage( self ):
        return self.age < 18
    
class Student( Person ):
    pass

albert = Student( "Albert", "Applebaum", 19 )
print( albert )
print( albert.underage() )

As you can see, the `Student` class inherits all properties and methods of the class `Person`.

### Extending and overriding

To extend a subclass with new methods, you can just define the new methods for the subclass. If you define methods that already exist in the parent class (or "superclass"), they "override" the parent class methods, i.e., they use the new method as specified by the subclass.

Often, when you override a method, you still want to use the method of the parent class. For instance, if the `Student` class needs a list of courses in which the student is enrolled, the course list must be initialized as an empty list in the `__init__()` method. Yet if I override the `__init__()` method, the student's name and age are no longer initialized, unless I make sure that they are. You can make a copy of the `__init__()` method for `Person` into `Student` and adapt that copy, but it is better to actually call the `__init__()` method of `Person` inside the `__init__()` method of `Student`. That way, should the `__init__()` method of `Person` change, there is no need to update the `__init__()` method of `Student`.

There are two ways of calling a method of another class: by using a "class call", or by using the `super()` method.

A class call entails that a method is called using the syntax `<classname>.<method>()`. So, to call the `__init__()` method of `Person`, I can write `Person.__init__()`. I am not limited to calling methods of the superclass this way; I can call methods of any class. Since such a call is not a regular method call, you *have* to supply `self` as an argument. So, for the code above, to call the `__init__()` method of `Person` from the `__init__()` method of `Student`, you write `Person.__init__( self, firstname, lastname, age )`.

Using `super()` means that you can directly refer to the superclass of a class by using the standard function `super()`, without knowing the name of the superclass. So to call the `__init__()` method of the superclass of `Student`, I can write `super().__init__()`. You do *not* supply `self` as the first argument if you use `super()` like this. So, for the code above, to call the `__init__()` method of `Person` from the `__init__()` method of `Student`, you write `super().__init__( firstname, lastname, age )`.

Of these two approaches, I prefer the use of `super()`, but only in this specific way: to call the immediate superclass in single-class inheritance. `super()` can be called in different ways and has a few intricacies, which I will get to below.

In the code below, the class `Student` gets two new attributes: a program and a course list. The method `__init__()` gets overridden to create these new attributes, but also calls the `__init__()` method of `Person`. `Student` gets a new method, `enroll()`, to add courses to the course list. Finally, as a demonstration I overrode the method `underage()` to make students underage when they are not 21 yet (sorry about that).

In [ ]:
class Person:
    def __init__( self, firstname, lastname, age ):
        self.firstname = firstname
        self.lastname = lastname
        self.age = age
    def __repr__( self ):
        return "{} {}".format( self.firstname, self.lastname )
    def underage( self ):
        return self.age < 18
    
class Student( Person ):
    def __init__( self, firstname, lastname, age, program ):
        super().__init__( firstname, lastname, age )
        self.courselist = []
        self.program = program
    def underage( self ):
        return self.age < 21
    def enroll( self, course ):
        self.courselist.append( course )

albert = Student( "Albert", "Applebaum", 19, "CSAI" )
print( albert )
print( albert.underage() )
print( albert.program )
albert.enroll( "Methods of Rationality" )
albert.enroll( "Defense Against the Dark Arts" )
print( albert.courselist )

### Multiple inheritance

You can create a class that inherits from multiple classes. This is called "multiple inheritance". You specify all the superclasses, with commas in between, between the parentheses of the class definition. The new class now forms a combination of all the superclasses.

When a method is called, to decide which method implementation to use, Python first checks whether it exists in the class for which the method is called itself. If it is not there, it checks all the superclasses, from left to right. As soon as it finds an implementation of the method, it will execute it.

If you want to call a method from a superclass, you have to tell Python which superclass you wish to call. You best do that directly with a class call. However, you can use `super()` for this too, but it is pretty tricky. You provide the order in which the classes should be checked as arguments to `super()`. However, the first argument is not checked by `super()` (I assume that it is supposed to be `self`).

It is something like this: You have three classes, `A`, `B`, and `C`. You create a new class `D` which inherits from all other three classes, by defining it as `class D( A, B, C )`. When in the `__init__()` method of `D` you want to call the `__init__()` methods of the three parent classes, you can call them using class calls as `A.__init__()`, `B.__init__()`, and `C.__init__()`. However, if you want to call the `__init__()` method of one of them, but you do not know exactly which, but you do know the order in which you want to check them (for instance, `B`, `C`, `A`), then you can call `super()` with `self` as the first argument and the other three classes in the order in which you want to check them (for instance `super( self, B, C, A ).__init__()`).

As I said, it is pretty tricky. Multiple inheritance is tricky anyway. My general recommendation is that you do not use it, unless there is really no way around it. Many object oriented languages do not even support multiple inheritance, and those that do tend to warn against using it.

So I am not even going to give an example of using multiple inheritance, and neither am I going to supply exercises for multiple inheritance. You should simply avoid using it, until you have a lot of experience with Python and object oriented programming. And by that time, you probably see ways of constructing your programs that do not need multiple inheritance at all.



---

## Interfaces

An interface is a class that specifies attributes and methods without an actual implementation of the methods. The idea is that subclasses implement the methods, while functions can be defined as working on the interface class, under the assumption that the methods will be filled in. Such functions can then be called with the subclasses.

For good understanding, it is probably better to give an example.

Suppose that I want to design an application that works with vehicles. Maybe it is a travel-planning application that calculates how to get from point A to point B. The application will have a map containing all possible points and connections between the points. It will also have a list of vehicles, with certain vehicles being restricted to specific points, and connecting only specific points (e.g., planes will only be available at airports, and only connect to specific other airports, while boats are only found in harbors and connect to specific other harbors). The application gets a start and end point as input, and provides a list of the sort: take the car to drive from start point to point X, take the plane to fly from point X to point Y, take the bus to drive from point Y to point Z, and then walk from point Z to the end point.

This class will need a definition of vehicles. To be able to come to an optimal travel plan, it must know for each vehicle at what points it is available, to what points it can travel, and the average speed of travel (so that you do not get a travel plan that says "walk from Amsterdam to Moscow"). It might also be a good idea to include a verb that is used when the plan refers to travel with a vehicle (e.g., "walk", "drive", or "fly"). You might need to think a lot about how to implement such vehicles. A possible approach is to supply each vehicle with a method that gets a point as argument and that returns whether or not the vehicle is available at that point, a method that gets a point as argument and that returns whether or not the vehicle travels to that point, a method that gets two points and returns the average speed of travel of the vehicle between those two points, and a method that returns the verb (I am not saying that this implementation is a good idea, just that it could potentially be used).

So you can implement a `Vehicle` class as follows:

In [ ]:
class Vehicle:
    def __init__( self ):
        self.startpoint = []
        self.endpoints = []
        self.verb = ""
        self.name = ""
    def __str__( self ):
        return self.name
    def isStartpoint( self, p ):
        return NotImplemented
    def isEndpoint( self, p ):
        return NotImplemented
    def travel_speed( self, p1, p2 ):
        return NotImplemented
    def travelVerb( self ):
        return NotImplemented    

A class like this is called an interface or "abstract class" (there are subtle differences between interfaces and abstract classes in computational theory, but for Python these do not matter). It is not to be used as a class of which you create instances, which is why all methods return `NotImplemented`. Instead, it is to be used as a template to inherit subclasses from, that will all create implementations for the predefined methods. This means that regardless which vehicle subclass you define later, you will always have to make sure the methods of the `Vehicle` class are implemented. So functions that make use of instances of subclasses of `Vehicle` may count on these methods being available. 

---

## What you learned

In this chapter, you learned about:

- Inheritance
- Overriding
- Class calls
- `super()`
- Multiple inheritance
- Interfaces

---

## Exercises

### Exercise 22.1

Below I give a `Rectangle` class that is created with the `x` and `y` coordinate of the top-left corner, a width `w`, and a height `h`. Now create a `Square` class that inherits as much as possible from the `Rectangle` class.

In [ ]:
# Square.
class Rectangle:
    def __init__( self, x, y, w, h ):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
    def __repr__( self ):
        return "[({},{}),w={},h={}]".format( self.x, self.y, self.w, self.h )
    def area( self ):
        return self.w * self.h
    def circumference( self ):
        return 2*(self.w + self.h)


### Exercise 22.2

A `Rectangle` and a `Square` can be considered shapes. There are, of course, different kinds of shapes which are defined differently, but share with rectangles and squares that they have an area and circumference. Define an interface class `Shape`, of which `Rectangle` and `Square` are sub(sub)classes. Also define a class `Circle` that you derive from `Shape`.

In [ ]:
# Shape.


### Exercise 22.3

In the Iterated Prisoner's Dilemma, two strategies play against each other over multiple rounds. Every round, the strategies can decide to either Coorperate (`C`) or Defect (`D`). If both cooperate, they both get 3 points. If both defect, they both get 1 point. If one cooperates and one defects, the one that defects gets 6 points, and the one that cooperates gets nothing. The goal for each strategy is to score as many points as possible.

Below a simple version of the Iterated Prisoner's Dilemma is coded. A strategy to play the game is defined by the class `Strategy`. The main loop lets two strategies play each other for 100 rounds (it is not hard to create a main loop that lets more than two strategies play each other in pairs, but that increases the size of the code quite a bit and is not important for the exercise). The `Strategy` class has not implemented the `choice()` method. To create a strategy, you inherit a new class from `Strategy`, and at least fill in the `choice()` method. Optionally you can also implement the `lastmove()` method, and extend the `__init__()` method.

Implement the following strategies: 
- `Random` just plays COOPERATE or DEFECT at random.
- `AlwaysDefect` always plays DEFECT.
- `TitForTat` starts with COOPERATE, then plays what the opponent played on the previous move.
- `TitForTwoTats` starts with two COOPERATEs, then plays DEFECT if the opponent played DEFECT on both the previous two moves, otherwise COOPERATEs.
- `Majority` starts with COOPERATE, then plays what the opponent played on the majority of the previous moves.

If you want to implement more strategies, be my guest. Test out some of the strategies against each other by filling in the assignments for `strategy1` and `strategy2` (do not forget to give them a name between the parentheses).

In [ ]:
# Iterated Prisoner's Dilemma
COOPERATE = 'C'
DEFECT = 'D'
ROUNDS = 100

class Strategy:
    def __init__( self, name="" ):
        self.name = name
        self.score = 0
    def choice( self ):
        # Should return COOPERATE or DEFECT
        return NotImplemented
    def lastmove( self, mymove, opponentmove ):
        # Gets passed the last move made, after a call of choice()
        pass
    def incscore( self, n ):
        self.score += n
    
strategy1 = Strategy()
strategy2 = Strategy()

for i in range( ROUNDS ):
    c1 = strategy1.choice()
    c2 = strategy2.choice()
    if c1 == c2:
        strategy1.incscore( 3 if c1 == COOPERATE else 1 )
        strategy2.incscore( 3 if c2 == COOPERATE else 1 )
    else:
        strategy1.incscore( 0 if c1 == COOPERATE else 6 )
        strategy2.incscore( 0 if c2 == COOPERATE else 6 )
    strategy1.lastmove( c1, c2 )
    strategy2.lastmove( c2, c1 )
        
print( "End score of", strategy1.name, "is", strategy1.score )
print( "End score of", strategy2.name, "is", strategy2.score )